<a href="https://colab.research.google.com/github/shravan002/Top-up-Strategy/blob/master/rough_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id': '14cjpDwnqTpKRTFJO9o8odOg-k0ckqps7'})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
#extract from zip file
import zipfile
import io
zf = zipfile.ZipFile(io.BytesIO(uploaded['lending-club-loan-data.zip']), "r")
zf.extractall()

In [ ]:
#Copy dataset in excel sheet

wb = op.load_workbook('pandas_openpyxl.xlsx')
ws = wb.get_sheet_by_name('Sheet2')
for r in dataframe_to_rows(df):
      ws.append(r)

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
A, A_count = np.unique(df['bad_acc'], return_counts=True)
print('A : ', A)
print('A_count : ', A_count)

In [ ]:
values, count = np.unique(df['issue_y_band1'], return_counts=True)

dfTemp = pd.DataFrame()
dfTemp['Values'] = values
dfTemp['Count'] = count
dfTemp.head(50)

In [ ]:
#Selecting multiple rows
import pandas as pd 
  
# making data frame from csv file 
data = pd.read_csv("nba.csv", index_col ="Name") 
  
# retrieving multiple rows by loc method 
first = data.loc[["Avery Bradley", "R.J. Hunter"]] 
print(first)

In [ ]:
# making data frame from csv file 
data1 = pd.read_csv("loan.csv", index_col ="loan_status") 
  
# retrieving multiple rows by loc method 
df1 = data1.loc[["Current"]] 


In [ ]:
dft = df[(df.term == ' 60 months') & (df.issue_y == '2014')]

In [ ]:
i = 1
for col in df1.columns:
  print(i,":", col)
  i =i+1

In [ ]:
x = []
x = np.percentile(df.int_rate, [0, 25, 50, 75, 100])

#Generate Bands according to Total income
inc_bands = [x[0], x[1], x[2], x[3], x[4]]
inc_bands_name =[1, 2, 3, 4]
df['inc_bandX'] = pd.cut(df['int_rate'], inc_bands, labels = inc_bands_name)

In [ ]:
#Delete a column
del df['band']
df = df.drop(['inc_band2'], axis=1)

In [ ]:
outliers = []
def detect_outliers(dataX):

  threshold = 3
  mean = np.mean(dataX)
  std = np.std(dataX)

  for i in dataX:
    z_score = (i - mean)/std
    if np.abs(z_score) > threshold:
      outliers.append(i)
  outliers.sort()
  print(outliers)

detect_outliers(df['total_inc'])

In [ ]:
dft = df[(df.term == ' 60 months') & (df.issue_y == '2014')]
dft.head() 

# print("Median: ", df['total_inc'].median())
# print("Mean:   ", df['total_inc'].mean())
# print("Max:    ", df['total_inc'].max())
# print("Min:    ", df['total_inc'].min())
# df['total_inc'].quantile([.25, .50, .75])

df['total_inc'].describe()

In [ ]:
data.info()

In [ ]:
#pip install pandas-profiling
from pandas_profiling import ProfileReport

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
profile = ProfileReport(dfp, minimal=True)
profile.to_file(output_file="output.html")

In [ ]:
profile = ProfileReport(dfp, title='Loans_strategy', html = {'style' : {'full_width': True}})

In [ ]:

#Removing unnecessary space before term string
df['term'] = df['term'].replace(' 36 months', 36)
df['term'] = df['term'].replace(' 60 months', 60)

#Converting issue date to string
df["issue_d"]= df["issue_d"].astype(str)

#Finding issue Year
df["issue_y"]= df["issue_d"].str.slice(4, 8, 1)
df["issue_y"] = df["issue_y"].astype(float)

#Finding issue Month
df["issue_m"] = df["issue_d"].str.slice(0, 3, 1)
df['issue_m'].replace(to_replace = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                           value = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                         inplace = True)

#Selecting last five year data
indexNames = df[(df['issue_y'] == 2007) | (df['issue_y'] == 2008)| (df['issue_y'] == 2009)| (df['issue_y'] == 2010)| (df['issue_y'] == 2011)| (df['issue_y'] == 2012)].index
df.drop(indexNames , inplace=True)

#Finding Total number of rows 
total_rows_prev = total_rows
total_rows = len(df.axes[0])
print('Eliminated Acc before 2013:', total_rows_prev - total_rows, total_rows)

#Eliminating Acc Issue Year is 2013 or 2014 of 36 months term 
indexNames = df[(df['term'] == 36 ) & ((df['issue_y'] == 2013) | (df['issue_y'] ==  2014))].index
df.drop(indexNames , inplace=True)

#Finding Total number of rows 
total_rows_prev = total_rows
total_rows = len(df.axes[0])
print('Issue_y 2013 or 2014 of 36 m:', total_rows_prev - total_rows, total_rows)

In [ ]:

#Calculating Term left
df['term_left'] = df['term'] - (((2019 - df['issue_y']) * 12) - df['issue_m'])

#Calculate percentage of Pay down
df['%pay_down'] = df['total_pymnt'] / df['loan_amnt'] * 100

#Generate Band according Percentage of Pay down
pay_down_bands = [0, 20, 40, 60, 80, 100]
pay_down_bands_name =[1, 2, 3, 4, 5]
df['%pay_down_band'] = pd.cut(df['%pay_down'], pay_down_bands, labels = pay_down_bands_name)

#Generate Bands according to Term Left
term_left_band = [0, 10, 20, 30, 40, 55]
term_left_bands_name = [1, 2, 3, 4, 5]
df['term_left_band'] = pd.cut(df['term_left'], term_left_band, labels = term_left_bands_name)

In [ ]:
df1['Bad_rate'] = df1['Charged Off1'] / total_rows * 100
df1['Deli_rate_exc_grc_prd'] = df1['Deli_exc_grc_prd'] / total_rows *100
df1['Deli_rate_inc_grc_prd'] = df1['Deli_inc_grc_prd'] / total_rows *100